# Localize and undrift 
This notebook will run picasso_addon.localize.main() on a input *.ome.tif file or input *_locs.hdf5 file:
*  It automatically sets a minimal net-gradient for spot detection
*  It applies a least square 2D Gaussian fit to the spots using picasso.localize
*  It undrifts the resulting localizations using the picasso.render RCC undrift option

We therefore:

1. Define the full paths to the *ome.tif files
2. Set the execution parameters
3. Run picasso_addon.localize.main() for all datasets

As a result files with extension *_locs.hdf5 (i.e. localized) and  *_render.hdf5 (i.e undrifted) will be created in the same folder as the *.ome.tif file.

In [2]:
import os
import traceback
import importlib
import matplotlib.pyplot as plt

import picasso.io as io
import picasso.localize
import picasso_addon.localize as localize

importlib.reload(localize)

<module 'picasso_addon.localize' from 'C:/Users/flori/Documents/mpi/repos/picasso_addon\\picasso_addon\\localize.py'>

### 1. Define the full paths to the *ome.tif files

In [ ]:
dir_names=[]
dir_names.extend([r'C:\Data\p06.SP-tracking\20-03-13_c-series_SDS_L21\03_id162_Pm2-40nM_L21_1'])

file_names=[]
file_names.extend(['03_id162_Pm2-40nM_L21_1_MMStack_Pos0.ome_locs.hdf5'])

### 2. Set the execution parameters

In [ ]:
### Valid for all evaluations
params_all={'localize':False,
            }

### Exceptions
params_special={}

For all possible parameters please run  `help(localize.main)` or readthedocs. If not stated otherwise standard values are used (indicated in brackets). Note that if a *_locs.hdf5 file is given as input we have to set `'localize':False` and only undrifting is performed!

In [4]:
help(localize.main)

Help on function main in module picasso_addon.localize:

main(file, info, path, **params)
    Localize movie (least squares, GPU fitting if available) and undrift resulting localizations using rcc.
     
    Args:
        file(picasso.io):          Either raw movie loaded with picasso.io.load_movie() or_locs.hdf5 loaded with picasso.io.load_locs()
        info(list(dicts)):         Info to raw movie/_locs.hdf5 loaded with picasso.io.load_movie() or picasso.io.load_locs()
    
    Keyword Arguments:
        localize(bool=True)        Localize raw movie (see picasso.localize)
        baseline(int=70):          Camera spec. baseline (see picasso.localize).
        gain(float=1):             Camera spec. EM gain (see picasso.localize)
        sensitivity(float=0.56):   Camera spec. sensitivity (see picasso.localize)
        qe(float=0.82):            Camera spec. sensitivity (see picasso.localize)
        box(int=5):                Box length (uneven!) of fitted spots (see picasso.localize

### 3. Run picasso_addon.localize.main() for all datasets

In [ ]:
failed_path=[]
for i in range(0,len(file_names)):
    ### Create path
    path=os.path.join(dir_names[i],file_names[i])
    ### Set paramters for each run
    params=params_all.copy()
    for key, value in params_special.items():
        params[key]=value[i]
    
    ### Run main function
    try:
        try:
            file,info=io.load_movie(path) # Load movie
        except:
            file,info=io.load_locs(path) # Load _locs            
        out=localize.main(file,info,path,**params)    
    
    except Exception:
        traceback.print_exc()
        failed_path.extend([path])

print()    
print('Failed attempts: %i'%(len(failed_path)))
